In [ ]:
project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot

import torchvision
from torchvision.transforms import Compose, ToTensor, ToPILImage, Normalize

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import random


from InformationPlane import TensorKernel, MatrixBasedRenyisEntropy, RKHSMatrixOptimizer, InformationPlane
from models.Paper import CNN
from datasets.MyMNIST import MyMNIST


### Load MNIST dataset

In [ ]:
transformToTensor = Compose([ ToTensor(), Normalize((0.1307,), (0.3081,))])
# # transformToTensor = Compose([ ToTensor()])

transformToImage = Compose([  Normalize((-0.1307/0.3081,), (1/0.3081,)), ToPILImage()])
# # transformToImage = Compose([ ToPILImage()])

dataset = torchvision.datasets.MNIST("../datasets/MNIST/", train=True, download=True, transform=transformToTensor)

dataset2 = MyMNIST()
dataset2.eval()

dataloader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0)
dataloader2 = DataLoader(dataset2, batch_size=100, shuffle=False, num_workers=0)

### Preparing test


In [ ]:
for input, label in dataloader:
    plt.imshow(transformToImage(input[0]), cmap="gray")
    plt.show()
    break

### Information plane test

In [ ]:
class RKHSMatrixOptimizer():
    def __init__(self, beta=0.5):
        if not(0 <= beta <= 1):
            raise Exception('beta must be in the range [0, 1]')

        self.beta = beta
        self.sigma = None
        self.sigma_tmp = [] #Just for saving sigma values

    # Temporal, just for testing
    def getSigmaValues(self):
        return self.sigma_tmp

    def getSigma(self):
        return self.sigma

    '''
        @param The output of a specific layer
        @param label_kernel_matrix
        @param n_sigmas
    '''
    def step(self, layer_output, Ky, n_sigmas=75):
        sigma_t = self.optimize(layer_output, Ky, n_sigmas)
        self.sigma = ( (self.beta*sigma_t) + ((1-self.beta)*self.sigma) ) if not(self.sigma is None) else sigma_t
        return self.getSigma()

    '''
        This function is used in orter to obtain the optimal kernel width for
        an T DNN layer

        @param layer_output
        @param n_sigmas: number of possible sigma values

        [Descripción del procedimiento]
    '''
    def optimize(self, layer_output, Ky, n_sigmas):
        distance = torch.cdist(layer_output, layer_output)
        mean_distance = distance[distance != 0].mean().detach().cpu()
        sigma_values = np.arange(mean_distance*0.1, mean_distance*10, (mean_distance*10 - mean_distance*0.1)/n_sigmas)

        Kt = list( map(lambda sigma: TensorKernel.RBF(layer_output, sigma).detach(), sigma_values.tolist()) )
        loss = np.array( list( map(lambda k: self.kernelAligmentLoss(k, Ky), Kt) ) )
        self.sigma_tmp.append(sigma_values[ np.argwhere(loss == loss.max()).item(0) ])
        return sigma_values[ np.argwhere(loss == loss.max()).item(0) ]

    '''
        Kernel Aligment Loss Function.

        This function is used in order to obtain the optimal sigma parameter from
        RBF kernel.  
    '''
    def kernelAligmentLoss(self, x, y):
        return (torch.sum(x*y))/(torch.norm(x) * torch.norm(y))


'''
    Information Plane is just generated using evaluation process.
'''
class InformationPlane(torch.nn.Module):
    '''
        @param input_kernel: preprocessed input kernel matrix
        @param input_kernel: preprocessed label kernel matrix
        @param sigma_values: number of possible sigma values for optimizing process.
        @param step: indicates the number of step for reducing the number of possible sigma values
    '''
    def __init__(self, beta=0.5):
        super(InformationPlane, self).__init__()

        self.sigma_optimizer = RKHSMatrixOptimizer(beta)
        self.Ixt = []
        self.Ity = []

        self.input_batch = None
        self.label_batch = None

    '''
        It's necessary to update the X and Y, input and label, in each iteration.

        @param input: batch with the original input
        @param label: label of the data
    '''
    def setInputLabel(self, input, label):
        self.input_batch = input
        self.label_batch = label

    '''
        @param beta regularizer term to stabilize the optimal sigma value across the previous iteration
        
        @return mutual information with label {I(X,T), I(T,Y)}
    '''
    def forward(self, x):
        if self.training:
            return x

        original_shape = x.shape
        x = x.flatten(1)
        label_kernel_matrix = TensorKernel.RBF(self.label_batch, 0.1)
        self.sigma_optimizer.step(x, label_kernel_matrix, 75)

        A = MatrixBasedRenyisEntropy.tensorRBFMatrix(x, self.sigma_optimizer.getSigma()).detach()
        Ay = MatrixBasedRenyisEntropy.tensorRBFMatrix(self.label_batch, 0.1).detach()
        Ax = MatrixBasedRenyisEntropy.tensorRBFMatrix(self.input_batch.flatten(1), 8).detach()

        self.Ixt.append(MatrixBasedRenyisEntropy.mutualInformation(Ax, A)) 
        self.Ity.append(MatrixBasedRenyisEntropy.mutualInformation(A, Ay))

        x = x.reshape(original_shape)
        return x
    
    # def forward(self, x, input, label):
    #     original_shape = x.shape
    #     x = x.flatten(1)
        
    #     label_kernel_matrix = TensorKernel.RBF(label, 0.1)
    #     self.sigma_optimizer.step(x, label_kernel_matrix, 75)

    #     A = MatrixBasedRenyisEntropy.tensorRBFMatrix(x, self.sigma_optimizer.getSigma()).detach()
    #     Ay = MatrixBasedRenyisEntropy.tensorRBFMatrix(label, 0.1).detach()
    #     Ax = MatrixBasedRenyisEntropy.tensorRBFMatrix(input.flatten(1), 8).detach()

    #     self.Ixt.append(MatrixBasedRenyisEntropy.mutualInformation(Ax, A)) 
    #     self.Ity.append(MatrixBasedRenyisEntropy.mutualInformation(A, Ay))

    #     x = x.reshape(original_shape)
    #     return x

    ''' 
        @return Mutual Information {I(X,T), I(T,Y)}
    '''
    def getMutualInformation(self):
        return self.Ixt, self.Ity


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1_IP = InformationPlane(beta=0.5)
        self.layer2_IP = InformationPlane(beta=0.5)
        self.layer3_IP = InformationPlane(beta=0.5)
        self.layer4_IP = InformationPlane(beta=0.5)
        self.fc_IP = InformationPlane(beta=0.5)


        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 4, 3, stride=1, padding=0),
            nn.ReLU(inplace=True),
            self.layer1_IP,     
            nn.BatchNorm2d(4)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(4, 8, 3, stride=1, padding=0),
            nn.ReLU(inplace=True),
            self.layer2_IP,
            nn.BatchNorm2d(8),      
            nn.MaxPool2d(2, stride=2, ceil_mode=True)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(8, 16, 3, stride=1, padding=0),
            nn.ReLU(inplace=True),
            self.layer3_IP,
            nn.BatchNorm2d(16),         
            nn.MaxPool2d(2, stride=2, ceil_mode=True)
        )

        self.fc = nn.Sequential(
            nn.Linear(400, 256),
            nn.ReLU(inplace=True),
            self.layer4_IP,
            nn.BatchNorm1d(256),
            nn.Linear(256, 10)
        )

        
        self.softmax = torch.nn.Softmax()

    def forward(self, x, label):
        [ip.setInputLabel(x, label) for ip in self.getInformationPlaneLayers()]

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        self.fc_IP(self.softmax(x))
        return x

    def getInformationPlaneLayers(self):
        return [self.layer1_IP, self.layer2_IP, self.layer3_IP, self.layer4_IP, self.fc_IP]

# Multilayer Perceptron

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        self.layer1_IP = InformationPlane(beta=0.5)
        self.layer2_IP = InformationPlane(beta=0.5)
        self.layer3_IP = InformationPlane(beta=0.5)
        self.layer4_IP = InformationPlane(beta=0.5)
        self.layer5_IP = InformationPlane(beta=0.5)

        self.layer1 = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(inplace=True),
            # self.layer1_IP,
            
        )

        self.layer2 = nn.Sequential(
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 20),
            nn.ReLU(inplace=True)
            # self.layer2_IP,
        )
        
        self.layer3 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.Linear(20, 20),
            nn.ReLU(inplace=True),
            # self.layer3_IP,
            
        )

        self.layer4 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.Linear(20, 20),
            nn.ReLU(inplace=True),
            # self.layer4_IP,
            
        )

        self.layer5 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.Linear(20, 10)
        )


        self.softmax = torch.nn.Softmax()

    def forward(self, x, label):
        # [ip.setInputLabel(x, label) for ip in self.getInformationPlaneLayers()]

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        # self.layer5_IP(self.softmax(x))

        return x

    def compute_mi(self, x, label):
        [ip.setInputLabel(x, label) for ip in self.getInformationPlaneLayers()]
        x = self.layer1(x)
        self.layer1_IP(x)
        x = self.layer2(x)
        self.layer2_IP(x)
        x = self.layer3(x)
        self.layer3_IP(x)
        x = self.layer4(x)
        self.layer4_IP(x)
        x = self.layer5(x)
        self.layer5_IP(self.softmax(x))

    def getInformationPlaneLayers(self):
        return [self.layer1_IP, self.layer2_IP, self.layer3_IP, self.layer4_IP, self.layer5_IP]

In [ ]:
# net = CNN().cuda()
net = MLP().cuda()

# Training

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import math

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.009, momentum=0.9)

def train():
    for epoch in range(1):  # loop over the dataset multiple times

        running_loss = 0.0
        net.train()
        for inputs, labels in tqdm(dataloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.flatten(1).cuda()
            # inputs = inputs.cuda()
            labels = labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            # outputs = net(inputs, one_hot(labels, num_classes=10).float())
            outputs = net(inputs, labels.reshape((len(labels), 1)).float())

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # for ip in net.getInformationPlaneLayers():
            #     Ixt, Ity = ip.getMutualInformation()
            #     if math.isnan(Ixt[-1]) or math.isnan(Ity[-1]):
            #         print("IP: Error!")
            #         return inputs, labels

            # print statistics
            # running_loss += loss.item()
            # if (i+1) % 25 == 0:    # print every 25 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #         (epoch + 1, i + 1, running_loss / 25))
            #     running_loss = 0.0
        
        # net.eval()
        # for inputs, labels in dataloader2:
        #     inputs = inputs.flatten(1).cuda()
        #     net.compute_mi(inputs, labels)

    print('Finished Training')

# inputs, labels = train()
train()

In [ ]:
ips = net.getInformationPlaneLayers()[0]
ips.eval()
print(ips.training)
ips.train()
print(ips.training)

In [ ]:
def moving_average(a, n=10) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
# mpl.style.use('seaborn')
plt.scatter(moving_average(net.layer1_IP.getMutualInformation()[0]), moving_average(net.layer1_IP.getMutualInformation()[1]), label="Layer 1")
plt.scatter(moving_average(net.layer2_IP.getMutualInformation()[0]), moving_average(net.layer2_IP.getMutualInformation()[1]), label="Layer 2")
plt.scatter(moving_average(net.layer3_IP.getMutualInformation()[0]), moving_average(net.layer3_IP.getMutualInformation()[1]), label="Layer 3")
plt.scatter(moving_average(net.layer4_IP.getMutualInformation()[0]), moving_average(net.layer4_IP.getMutualInformation()[1]), label="Layer 4")
# plt.scatter(moving_average(net.fc_IP.getMutualInformation()[0]), moving_average(net.fc_IP.getMutualInformation()[1]), label="Layer 5")
plt.scatter(moving_average(net.layer5_IP.getMutualInformation()[0]), moving_average(net.layer5_IP.getMutualInformation()[1]), label="Layer 5")
plt.xlabel("I(X, T)")
plt.ylabel("I(T, Y)")
plt.legend()
plt.show()

In [ ]:
net.layer1_IP.getMutualInformation()[0]
net.layer1_IP.getMutualInformation()[1]

In [ ]:
with plt.style.context('seaborn'):
    ips = net.getInformationPlaneLayers()
    # ips = [net.layer1_IP, net.layer2_IP, net.layer3_IP, net.layer4_IP, net.layer5_IP]
    # ips = [net.layer1_IP, net.layer2_IP, net.layer3_IP, net.layer4_IP, net.fc_IP]
    colors = ['Reds', 'Blues', 'binary', 'Greens', 'Oranges']

    fig = plt.figure(constrained_layout=False)
    gs1 = fig.add_gridspec(nrows=1, ncols=1, left=0.05, right=0.84, wspace=0.05)
    gs2 = fig.add_gridspec(nrows=1, ncols=5, left=0.85, right=0.95, wspace=0)
    f8_ax1 = fig.add_subplot(gs1[:, :])
    f8_ax1.set_xlabel("I(X, T)")
    f8_ax1.set_ylabel("I(T, Y)")

    for idx, ip in enumerate(ips):
        cmap = plt.cm.get_cmap(colors[idx])
        Ixt, Ity = ip.getMutualInformation()
        Ixt = moving_average(Ixt)
        Ity = moving_average(Ity)
        iterations = np.arange(len(Ixt))
        color = np.array([cmap(iterations[-1])])
        sc = f8_ax1.scatter(Ixt, Ity, c=iterations, vmin=0, vmax=iterations.max(), cmap=cmap, edgecolor=color)
        f8_ax1.scatter([], [], c=color, label="Layer {}".format(idx))

        f8_ax2 = fig.add_subplot(gs2[0, idx])
        cb = fig.colorbar(sc, cax=f8_ax2, pad=0)
        cb.set_ticks([])

    f8_ax1.legend()
    cb.set_ticks([0, iterations.max()])
    cb.set_label("Iterations", labelpad=-18)

    plt.show()

In [ ]:
from numpy import linalg as LA

ips = net.getInformationPlaneLayers()

Ixt_0, Ity_0 = ips[0].getMutualInformation()
Ixt_1, Ity_1 = ips[-1].getMutualInformation()

# print(Ixt_0)
# print(Ity_0)
# print("---"*10)
# print(Ixt_1)
# print(Ity_1)
labels_oh = one_hot(labels, num_classes=10).float()
labels_test = labels.reshape((len(labels), 1)).float()
# print(labels_test[0:100:10])
# print(torch.cdist(labels_test[0:100:10], labels_test[0:100:10]))
test = torch.cdist(labels_oh, labels_oh)
print(torch.unique(test))
test = torch.cdist(labels_test, labels_test)
print(torch.unique(test))

# print(labels_oh[0:100:10])
# print(torch.cdist(labels_oh[0:100:10], labels_oh[0:100:10]))
# print(torch.cdist(labels_oh, labels_oh))
kernel = TensorKernel.RBF(labels_test, 0.25)
# print(kernel)
print(torch.unique(kernel))

A = TensorKernel.RBF(labels_test[0:50:10], 0.1) / len(labels_test)
print(A)
# softmax = nn.Softmax()
# output = net(inputs, labels)


# test = InformationPlane(beta=0.5)

# outputs = net(inputs, inputs, one_hot(labels, num_classes=10).float())
# outputs = net.softmax(outputs)
# sigma = net.fc_IP.sigma_optimizer.getSigma()

# label = one_hot(labels, num_classes=10).float()
# # print(label)
# label_kernel_matrix = TensorKernel.RBF(label, 0.1)
# print(label_kernel_matrix)
# A = MatrixBasedRenyisEntropy.tensorRBFMatrix(outputs.flatten(1), sigma+3).detach()
# # print(A)
# Ay = MatrixBasedRenyisEntropy.tensorRBFMatrix(label, 0.1).detach()
# # print(Ay)
# Ax = MatrixBasedRenyisEntropy.tensorRBFMatrix(inputs.flatten(1), 8).detach()
# # print(Ax)
# # print(entropy(A))
# # print(MatrixBasedRenyisEntropy.mutualInformation(Ax, A))

# from scipy import linalg

# def entropy(A : np.array):
#         epsilon = 1e-6
#         w = linalg.eigh(A, eigvals_only=True)
#         # w, _ = LA.eig(A)
#         w += epsilon
#         print(w)
#         return -np.sum(w * np.log2(w))

# def testMutualInformation(Ax : np.array, Ay : np.array):
#         entropy_Ax = MatrixBasedRenyisEntropy.entropy(Ax)
#         entropy_Ay = entropy(Ay)
#         # print(entropy_Ax)
#         # print(entropy_Ay)
#         joint_entropy_AxAy = MatrixBasedRenyisEntropy.jointEntropy(Ax, Ay)
#         return (entropy_Ax + entropy_Ay - joint_entropy_AxAy)

# print(testMutualInformation(A, Ay))
# print(Ixt)

In [ ]:
from scipy import linalg
from numpy import linalg as LA

def RBF(x, sigma):
    distance = torch.pdist(x, x)
    return torch.exp(-(distance**2)/(2*(sigma**2))).cpu()

labels_oh = one_hot(labels, num_classes=10).float()
labels_test = labels.reshape((len(labels), 1)).float()

A = RBF(labels_test[5:10], 0.1) / len(labels_test[5:10])
print(labels_test[5:10])
print(A)
# w, _ = LA.eig(A)
# print(w)
w = linalg.eigh(A, eigvals_only=True)
print(w)
# print(len(w))
# print(np.log2(w))
# print(np.log2(w)*w)
# print(-np.sum(w * np.log2(w)))

# w += 1e-16
# print(w)
# print(-np.sum(w * np.log2(w)))

In [ ]:
# torch.save(labels, 'tests/labels.pt')
# torch.save(inputs, 'tests/inputs.pt')


In [ ]:
for idx, ip in enumerate(ips):
    plt.plot(ip.sigma_optimizer.getSigmaValues(), label="layer {}".format(idx+1))
    
plt.legend()
plt.show()

In [ ]:
a = torch.rand((3,10))

In [ ]:
print(a)

In [ ]:
print(torch.cdist(a,a))